In [1]:
import os
import numpy as np
from PIL import Image, ImageFile
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings

##### Get the Data and Load them and labels

In [2]:

# this might need to change based on your file structure
data_dir = '/Users/kaunli/Desktop/School/Machine Learning/Project/WasteClassData'

image_size = (128, 128)
categories = ['Hazardous', 'Non-Recyclable', 'Organic', 'Recyclable']
num_classes = len(categories)
allowed_extensions = ('.jpg', '.jpeg', '.png')

# allow partially corrupted images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# ignore warnings
warnings.filterwarnings("ignore", category=UserWarning, module="PIL.TiffImagePlugin")

In [3]:
# load the images
images = []
labels = []

for idx, main_category in enumerate(categories):
    main_path = os.path.join(data_dir, main_category, main_category)
    if not os.path.exists(main_path):
        print(f"Path not found: {main_path}")
        continue
    for subfolder in os.listdir(main_path):
        subfolder_path = os.path.join(main_path, subfolder)
        if not os.path.isdir(subfolder_path):
            continue
        for img_file in os.listdir(subfolder_path):
            if img_file.startswith('.') or not img_file.lower().endswith(allowed_extensions):
                continue
            try:
                img_path = os.path.join(subfolder_path, img_file)
                img = Image.open(img_path).convert('RGBA')  # handle transparency
                img = img.convert('RGB')  # convert to RGB
                img = img.resize(image_size)
                img_array = np.array(img)
                images.append(img_array)
                labels.append(idx)
            except Exception as e:
                print(f"Skipping corrupted or unsupported file: {img_path} ({e})")

images = np.array(images)
labels = np.array(labels)

print("Loaded images:", images.shape)
print("Loaded labels:", labels.shape)

Loaded images: (2884, 128, 128, 3)
Loaded labels: (2884,)


##### Split train, validation, and test sets with 80/10/10

In [4]:
X_train, X_temp, y_train, y_temp = train_test_split(
    images, labels, test_size=0.2, random_state=42, stratify=labels
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)


print("Train:", X_train.shape, y_train.shape)
print("Validation:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)

Train: (2307, 128, 128, 3) (2307,)
Validation: (288, 128, 128, 3) (288,)
Test: (289, 128, 128, 3) (289,)


##### One Hot Encoding

In [5]:
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

##### Data Augmentation

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)
test_generator = test_datagen.flow(X_test, y_test, batch_size=32, shuffle=False)

##### CNN Models

In [7]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.applications import VGG16

##### Baseline CNN (from scratch)

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning, module='keras')

baseline_model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# compile
baseline_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# baseline_model.summary()

In [ ]:
# train the model
history = baseline_model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,
    epochs=20,
    validation_data=val_generator,
    validation_steps=len(X_val) // 32
)
# evaluate the model
test_loss, test_accuracy = baseline_model.evaluate(test_generator, steps=len(X_test) // 32)
print(f'Test accuracy: {test_accuracy:.4f}')

**Interpret**
- Accuracy: about 32% on the test set
- Loss: 1.3553 is pretty high and not improving much by epochs